# Power BI Data Alignment Notebook

This notebook processes all player data from championship reports and prepares it for Power BI import.

## Overview
- Loads data from championship reports (Excel files)
- Calculates consistency scores, style fits, and Top 15s
- Merges with raw player stats
- Exports to Power BI-friendly formats (CSV/Excel)

## Output Structure
- **Fact Tables**: Player performance data
- **Dimension Tables**: Players, Teams, Conferences, Positions, Metrics


## 1. Setup & Configuration


In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
from pathlib import Path
import json
import sys
from openpyxl import load_workbook
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully")


In [ ]:
# ============================================================================
# PATH CONFIGURATION
# ============================================================================
# Update this path to match your system
BASE_DIR = Path("/Users/daniel/Documents/Smart Sports Lab/Football/Sports Data Campus/Portland Thorns/Data/Advanced Search")

# Input directories
CHAMPIONSHIP_REPORTS_DIR = BASE_DIR / "Championship Reports"
RAW_PLAYER_STATS_DIR = BASE_DIR / "Exports" / "Players Stats By Position"
TEAM_STATS_DIR = BASE_DIR.parent / "Brief Conferences"
HISTORICAL_DATA_DIR = BASE_DIR / "Exports" / "Past Seasons"

# Output directory for Power BI exports
OUTPUT_DIR = BASE_DIR / "Power_BI_Exports"
OUTPUT_DIR.mkdir(exist_ok=True)

# Configuration file
CONFIG_FILE = BASE_DIR / "Scripts" / "00_Keep" / "position_metrics_config.json"

print(f"📁 Base Directory: {BASE_DIR}")
print(f"📁 Output Directory: {OUTPUT_DIR}")


In [ ]:
# ============================================================================
# CONSTANTS & MAPPINGS
# ============================================================================

# Position profile mappings
POSITION_PROFILE_MAP = {
    'Hybrid CB': 'Center Back',
    'DM Box-To-Box': 'Centre Midfielder',
    'AM Advanced Playmaker': 'Attacking Midfielder',
    'Right Touchline Winger': 'Winger'
}

# Position to file prefix mapping
POSITION_TO_PREFIX = {
    'Hybrid CB': 'CB Hybrid',
    'DM Box-To-Box': 'DM Box-To-Box',
    'AM Advanced Playmaker': 'AM Advanced Playmaker',
    'Right Touchline Winger': 'W Touchline Winger',
    'Center Back': 'CB Hybrid',
    'Centre Midfielder': 'DM Box-To-Box',
    'Attacking Midfielder': 'AM Advanced Playmaker',
    'Winger': 'W Touchline Winger'
}

# Conferences
CONFERENCES = ['ACC', 'SEC', 'BIG10', 'BIG12', 'IVY']

# Position profiles
POSITION_PROFILES = ['Hybrid CB', 'DM Box-To-Box', 'AM Advanced Playmaker', 'Right Touchline Winger']

# Load configuration
with open(CONFIG_FILE, 'r') as f:
    CONFIG = json.load(f)

print("✅ Configuration loaded")


In [ ]:
def load_players_from_report(report_file, position_profile, base_dir):
    """
    Load players from a position profile sheet in a conference report.
    Handles merged headers (row 1 + row 2) and enriches with raw data.
    """
    try:
        wb = load_workbook(report_file, data_only=True)
        
        if position_profile not in wb.sheetnames:
            wb.close()
            return pd.DataFrame()
        
        ws = wb[position_profile]
        
        # Read headers from row 1 and row 2 (merged headers)
        headers = []
        last_header1 = None
        
        for col_idx in range(1, ws.max_column + 1):
            header1 = ws.cell(row=1, column=col_idx).value
            header2 = ws.cell(row=2, column=col_idx).value
            
            if header1:
                last_header1 = str(header1).strip()
            
            header1_str = last_header1 if last_header1 else None
            header2_str = str(header2).strip() if header2 else None
            
            # Combine headers intelligently
            if header1_str and header2_str:
                if header2_str.lower() in ['per 90', 'per90', '% better than position', '%', 'won, %', 'won %', 'accurate, %', 'accurate %']:
                    full_header = f"{header1_str} {header2_str}"
                elif header1_str.lower() in ['player', 'team', 'position', 'conference grade', 'power five grade', '2025 total score', 
                                              'previous year', 'previous score', 'change from previous', 'total minutes', 
                                              '% of team minutes', 'top 15s (power five)', 'seasons played', 'changed position']:
                    full_header = header1_str
                elif header1_str.lower() == header2_str.lower():
                    full_header = header1_str
                else:
                    full_header = f"{header1_str} {header2_str}".strip()
            elif header1_str:
                full_header = header1_str
            elif header2_str:
                full_header = header2_str
            else:
                full_header = f"Column_{col_idx}"
            
            headers.append(full_header)
        
        # Read data starting from row 3
        data = []
        for row_idx in range(3, ws.max_row + 1):
            row_data = []
            for col_idx in range(1, len(headers) + 1):
                cell_value = ws.cell(row=row_idx, column=col_idx).value
                row_data.append(cell_value)
            
            # Skip empty rows
            if not any(cell for cell in row_data if cell not in [None, '', ' ']):
                continue
            
            data.append(row_data)
        
        wb.close()
        
        if not data:
            return pd.DataFrame()
        
        # Create dataframe
        df = pd.DataFrame(data, columns=headers[:len(data[0])] if data else headers)
        
        # Get conference name from filename
        conference = report_file.stem.replace('Portland Thorns 2025 ', '').replace(' Championship Scouting Report', '')
        df['Conference'] = conference
        
        # Load raw data to enrich with percentage metrics
        file_prefix = POSITION_TO_PREFIX.get(position_profile)
        if file_prefix:
            raw_file = base_dir / "Exports" / "Players Stats By Position" / f"{file_prefix} {conference} 2025.xlsx"
            if raw_file.exists():
                try:
                    df_raw = pd.read_excel(raw_file)
                    if 'Player' in df_raw.columns and 'Player' in df.columns:
                        exclude_cols = ['Team', 'Position', 'Minutes played', 'Duration']
                        metric_cols = [col for col in df_raw.columns if col not in exclude_cols and col != 'Player']
                        df_raw_subset = df_raw[['Player'] + metric_cols].copy()
                        
                        # Normalize for matching
                        df['Player_normalized'] = df['Player'].astype(str).str.strip().str.lower()
                        df_raw_subset['Player_normalized'] = df_raw_subset['Player'].astype(str).str.strip().str.lower()
                        
                        df['Team_normalized'] = df['Team'].astype(str).str.strip().str.lower()
                        if 'Team' in df_raw_subset.columns:
                            df_raw_subset['Team_normalized'] = df_raw_subset['Team'].astype(str).str.strip().str.lower()
                            merge_on = ['Player_normalized', 'Team_normalized']
                        else:
                            merge_on = ['Player_normalized']
                        
                        # Merge
                        df_merged = df.merge(df_raw_subset[merge_on + metric_cols], on=merge_on, how='left', suffixes=('_report', ''))
                        
                        # Drop normalized columns
                        df_merged = df_merged.drop(columns=['Player_normalized'], errors='ignore')
                        if 'Team_normalized' in df_merged.columns:
                            df_merged = df_merged.drop(columns=['Team_normalized'], errors='ignore')
                        
                        df = df_merged
                except Exception as e:
                    print(f"     ⚠️  Could not load raw data: {e}")
        
        return df
        
    except Exception as e:
        print(f"  ⚠️  Error loading {position_profile} from {report_file.name}: {e}")
        return pd.DataFrame()

print("✅ Data loading function defined")


In [ ]:
def load_all_players_from_reports(base_dir):
    """
    Load all players from all championship reports across all conferences and positions.
    """
    all_players = []
    
    reports_dir = base_dir / "Championship Reports"
    
    for conference in CONFERENCES:
        report_file = reports_dir / f"Portland Thorns 2025 {conference} Championship Scouting Report.xlsx"
        
        if not report_file.exists():
            print(f"⚠️  Report not found: {report_file.name}")
            continue
        
        print(f"\n📊 Loading {conference}...")
        
        for position_profile in POSITION_PROFILES:
            df = load_players_from_report(report_file, position_profile, base_dir)
            
            if not df.empty:
                df['Position_Profile'] = position_profile
                all_players.append(df)
                print(f"  ✅ {position_profile}: {len(df)} players")
    
    if all_players:
        combined_df = pd.concat(all_players, ignore_index=True)
        print(f"\n✅ Total players loaded: {len(combined_df)}")
        return combined_df
    else:
        return pd.DataFrame()

print("✅ Load all players function defined")


## 3. Load All Data


In [ ]:
# Load all players from championship reports
df_all_players = load_all_players_from_reports(BASE_DIR)

print(f"\n📊 Data Summary:")
print(f"   Total Players: {len(df_all_players)}")
if not df_all_players.empty:
    print(f"   Conferences: {df_all_players['Conference'].nunique()}")
    print(f"   Position Profiles: {df_all_players['Position_Profile'].nunique()}")
    print(f"\n   Columns: {len(df_all_players.columns)}")


## 4. Data Cleaning & Standardization


In [ ]:
# Standardize column names for Power BI
def standardize_column_names(df):
    """
    Standardize column names to be Power BI friendly (no special characters, spaces replaced with underscores).
    """
    df = df.copy()
    
    # Create mapping of old to new column names
    column_mapping = {}
    
    for col in df.columns:
        new_col = str(col)
        # Replace spaces with underscores
        new_col = new_col.replace(' ', '_')
        # Remove special characters
        new_col = ''.join(c if c.isalnum() or c == '_' else '' for c in new_col)
        # Remove multiple underscores
        new_col = '_'.join(filter(None, new_col.split('_')))
        # Ensure it doesn't start with a number
        if new_col and new_col[0].isdigit():
            new_col = 'Col_' + new_col
        
        column_mapping[col] = new_col
    
    df = df.rename(columns=column_mapping)
    return df

# Clean and standardize
if not df_all_players.empty:
    df_cleaned = standardize_column_names(df_all_players)
    
    # Fill missing values with 0 for numeric columns
    numeric_cols = df_cleaned.select_dtypes(include=[np.number]).columns
    df_cleaned[numeric_cols] = df_cleaned[numeric_cols].fillna(0)
    
    # Fill missing string values with empty string
    string_cols = df_cleaned.select_dtypes(include=['object']).columns
    df_cleaned[string_cols] = df_cleaned[string_cols].fillna('')
    
    print("✅ Data cleaned and standardized")
    print(f"   Columns: {len(df_cleaned.columns)}")
else:
    df_cleaned = pd.DataFrame()
    print("⚠️  No data to clean")


## 5. Create Dimension Tables


In [ ]:
# Create dimension tables for Power BI star schema

if not df_cleaned.empty:
    # Dimension: Players
    dim_players = df_cleaned[[
        'Player', 'Team', 'Conference', 'Position_Profile'
    ]].drop_duplicates().copy()
    dim_players['Player_ID'] = range(1, len(dim_players) + 1)
    dim_players = dim_players[['Player_ID', 'Player', 'Team', 'Conference', 'Position_Profile']]
    
    # Dimension: Teams
    dim_teams = df_cleaned[['Team', 'Conference']].drop_duplicates().copy()
    dim_teams['Team_ID'] = range(1, len(dim_teams) + 1)
    dim_teams = dim_teams[['Team_ID', 'Team', 'Conference']]
    
    # Dimension: Conferences
    dim_conferences = pd.DataFrame({
        'Conference_ID': range(1, len(CONFERENCES) + 1),
        'Conference': CONFERENCES,
        'Conference_Name': [c.replace('BIG10', 'Big Ten').replace('BIG12', 'Big 12') for c in CONFERENCES]
    })
    
    # Dimension: Position Profiles
    dim_positions = pd.DataFrame({
        'Position_Profile_ID': range(1, len(POSITION_PROFILES) + 1),
        'Position_Profile': POSITION_PROFILES,
        'Internal_Position': [POSITION_PROFILE_MAP.get(p, p) for p in POSITION_PROFILES]
    })
    
    print("✅ Dimension tables created")
    print(f"   Players: {len(dim_players)} rows")
    print(f"   Teams: {len(dim_teams)} rows")
    print(f"   Conferences: {len(dim_conferences)} rows")
    print(f"   Positions: {len(dim_positions)} rows")
else:
    print("⚠️  No data to create dimension tables")


In [ ]:
# Create fact tables with player performance data

if not df_cleaned.empty and 'dim_players' in locals():
    # Merge with dimension tables to get IDs
    df_fact = df_cleaned.merge(
        dim_players[['Player', 'Team', 'Conference', 'Position_Profile', 'Player_ID']],
        on=['Player', 'Team', 'Conference', 'Position_Profile'],
        how='left'
    )
    
    df_fact = df_fact.merge(
        dim_teams[['Team', 'Conference', 'Team_ID']],
        on=['Team', 'Conference'],
        how='left'
    )
    
    df_fact = df_fact.merge(
        dim_conferences[['Conference', 'Conference_ID']],
        on='Conference',
        how='left'
    )
    
    df_fact = df_fact.merge(
        dim_positions[['Position_Profile', 'Position_Profile_ID']],
        on='Position_Profile',
        how='left'
    )
    
    # Fact table: Player Performance (detailed)
    fact_player_performance = df_fact.copy()
    
    # Fact table: Player Summary (aggregated scores)
    summary_cols = [
        'Player_ID', 'Team_ID', 'Conference_ID', 'Position_Profile_ID',
        'Player', 'Team', 'Conference', 'Position_Profile'
    ]
    
    # Add score columns if they exist
    score_cols = [col for col in df_fact.columns if any(x in col.lower() for x in ['score', 'grade', 'consistency', 'style_fit', 'top_15'])]
    summary_cols.extend(score_cols)
    
    # Add base info columns
    base_cols = [col for col in df_fact.columns if any(x in col.lower() for x in ['minutes', 'seasons', 'previous', 'change'])]
    summary_cols.extend(base_cols)
    
    # Keep only columns that exist
    summary_cols = [col for col in summary_cols if col in df_fact.columns]
    
    fact_player_summary = df_fact[summary_cols].copy()
    
    print("✅ Fact tables created")
    print(f"   Player Performance: {len(fact_player_performance)} rows, {len(fact_player_performance.columns)} columns")
    print(f"   Player Summary: {len(fact_player_summary)} rows, {len(fact_player_summary.columns)} columns")
else:
    print("⚠️  No data to create fact tables")


## 7. Export to Power BI Formats


In [ ]:
# Export all tables to CSV and Excel for Power BI

if not df_cleaned.empty:
    # Export dimension tables
    dim_players.to_csv(OUTPUT_DIR / "dim_players.csv", index=False)
    dim_teams.to_csv(OUTPUT_DIR / "dim_teams.csv", index=False)
    dim_conferences.to_csv(OUTPUT_DIR / "dim_conferences.csv", index=False)
    dim_positions.to_csv(OUTPUT_DIR / "dim_positions.csv", index=False)
    
    print("✅ Dimension tables exported to CSV")
    
    # Export fact tables
    if 'fact_player_performance' in locals():
        fact_player_performance.to_csv(OUTPUT_DIR / "fact_player_performance.csv", index=False)
        print(f"✅ Fact table (performance) exported: {len(fact_player_performance)} rows")
    
    if 'fact_player_summary' in locals():
        fact_player_summary.to_csv(OUTPUT_DIR / "fact_player_summary.csv", index=False)
        print(f"✅ Fact table (summary) exported: {len(fact_player_summary)} rows")
    
    # Also export to Excel with multiple sheets
    with pd.ExcelWriter(OUTPUT_DIR / "Power_BI_Data.xlsx", engine='openpyxl') as writer:
        dim_players.to_excel(writer, sheet_name='dim_players', index=False)
        dim_teams.to_excel(writer, sheet_name='dim_teams', index=False)
        dim_conferences.to_excel(writer, sheet_name='dim_conferences', index=False)
        dim_positions.to_excel(writer, sheet_name='dim_positions', index=False)
        
        if 'fact_player_performance' in locals():
            fact_player_performance.to_excel(writer, sheet_name='fact_player_performance', index=False)
        
        if 'fact_player_summary' in locals():
            fact_player_summary.to_excel(writer, sheet_name='fact_player_summary', index=False)
    
    print(f"\n✅ All data exported to: {OUTPUT_DIR}")
    print(f"   - CSV files for each table")
    print(f"   - Excel file: Power_BI_Data.xlsx (all tables)")
else:
    print("⚠️  No data to export")


## 8. Data Summary & Validation


In [ ]:
# Display summary statistics

if not df_cleaned.empty:
    print("\n" + "="*70)
    print("DATA SUMMARY")
    print("="*70)
    
    print(f"\n📊 Total Players: {len(df_cleaned)}")
    print(f"📊 Unique Players: {df_cleaned['Player'].nunique() if 'Player' in df_cleaned.columns else 'N/A'}")
    print(f"📊 Conferences: {df_cleaned['Conference'].nunique() if 'Conference' in df_cleaned.columns else 'N/A'}")
    print(f"📊 Position Profiles: {df_cleaned['Position_Profile'].nunique() if 'Position_Profile' in df_cleaned.columns else 'N/A'}")
    
    if 'Conference' in df_cleaned.columns:
        print("\n📊 Players by Conference:")
        print(df_cleaned['Conference'].value_counts())
    
    if 'Position_Profile' in df_cleaned.columns:
        print("\n📊 Players by Position:")
        print(df_cleaned['Position_Profile'].value_counts())
    
    # Check for missing values in key columns
    key_cols = ['Player', 'Team', 'Conference', 'Position_Profile']
    print("\n📊 Missing Values Check:")
    for col in key_cols:
        if col in df_cleaned.columns:
            missing = df_cleaned[col].isna().sum()
            print(f"   {col}: {missing} missing ({missing/len(df_cleaned)*100:.1f}%)")
    
    print("\n" + "="*70)
else:
    print("⚠️  No data available for summary")


In [ ]:
# Display sample data

if not df_cleaned.empty:
    print("\n📋 Sample Data (first 5 rows):")
    display_cols = ['Player', 'Team', 'Conference', 'Position_Profile']
    display_cols = [col for col in display_cols if col in df_cleaned.columns]
    
    if display_cols:
        print(df_cleaned[display_cols].head())
    else:
        print(df_cleaned.head())
else:
    print("⚠️  No data to display")


## 9. Column Documentation


In [ ]:
# Generate column documentation

if not df_cleaned.empty:
    column_docs = []
    
    for col in df_cleaned.columns:
        doc = {
            'Column_Name': col,
            'Data_Type': str(df_cleaned[col].dtype),
            'Non_Null_Count': df_cleaned[col].notna().sum(),
            'Null_Count': df_cleaned[col].isna().sum(),
            'Unique_Values': df_cleaned[col].nunique() if df_cleaned[col].dtype == 'object' else 'N/A'
        }
        
        if df_cleaned[col].dtype in ['int64', 'float64']:
            doc['Min'] = df_cleaned[col].min()
            doc['Max'] = df_cleaned[col].max()
            doc['Mean'] = df_cleaned[col].mean()
        
        column_docs.append(doc)
    
    df_column_docs = pd.DataFrame(column_docs)
    df_column_docs.to_csv(OUTPUT_DIR / "column_documentation.csv", index=False)
    df_column_docs.to_excel(OUTPUT_DIR / "column_documentation.xlsx", index=False)
    
    print("✅ Column documentation exported")
    print(f"   File: {OUTPUT_DIR / 'column_documentation.csv'}")
else:
    print("⚠️  No data for documentation")


## 10. Power BI Import Instructions

### Steps to Import into Power BI:

1. **Open Power BI Desktop**

2. **Import Data**:
   - Click "Get Data" → "Text/CSV" or "Excel"
   - Navigate to the `Power_BI_Exports` folder
   - Import each CSV file OR import the Excel file `Power_BI_Data.xlsx`

3. **Set Up Relationships**:
   - `fact_player_performance` → `dim_players` (on Player_ID)
   - `fact_player_performance` → `dim_teams` (on Team_ID)
   - `fact_player_performance` → `dim_conferences` (on Conference_ID)
   - `fact_player_performance` → `dim_positions` (on Position_Profile_ID)
   - Same relationships for `fact_player_summary`

4. **Create Measures** (examples):
   - Average Total Score: `AVERAGE(fact_player_summary[Total_Score])`
   - Total Players: `COUNTROWS(dim_players)`
   - Players by Conference: Group by Conference

5. **Refresh Schedule**:
   - Set up scheduled refresh after running this notebook
   - Update data source paths if needed

### File Locations:
- **CSV Files**: `Power_BI_Exports/*.csv`
- **Excel File**: `Power_BI_Exports/Power_BI_Data.xlsx`
- **Documentation**: `Power_BI_Exports/column_documentation.csv`
